# Import libraries

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from typing import Dict

from torch import nn, optim, Tensor, torch
from torch_geometric.utils.num_nodes import maybe_num_nodes

In [ ]:
from data_loader.data_loader import UserUserDataset
from light_gcn.light_gcn import LightGCN, bpr_loss, get_user_positive_items, RecallPrecision_ATk, NDCGatK_r, get_metrics, evaluation

In [2]:
torch.__version__

'2.1.1+cu118'

In [3]:
postings_1 = pd.read_csv("../data/Postings_01052019_15052019.csv", sep = ';')
postings_2 = pd.read_csv("../data/Postings_16052019_31052019.csv", sep = ';')
votes_1 = pd.read_csv("../data/Votes_01052019_15052019.csv", sep = ';')
votes_2 = pd.read_csv("../data/Votes_16052019_31052019.csv", sep = ';')
postings = pd.concat([postings_1, postings_2])
votes = pd.concat([votes_1, votes_2])

datasets_dict = {
    "postings": postings,
    "votes": votes
}

In [ ]:
uu_dataset = UserUserDataset(datasets_dict = datasets_dict, verbose = True)

In [ ]:
train_edge_index, val_edge_index, test_edge_index = uu_dataset.get_train_test_val_split(val_split_ratio = 0.2, test_split_ratio = 0.5, random_state = 1, sparse = False)

In [ ]:
source, target, negative_target = uu_dataset.get_negative_samples(train_edge_index)

In [15]:
user_indices, pos_item_indices, neg_item_indices = uu_dataset.get_random_mini_batches(512)

In [18]:
num_users, num_articles = uu_dataset.num_users, uu_dataset.num_articles
model = LightGCN(num_users, num_articles)

In [21]:
# define constants
ITERATIONS = 10000
BATCH_SIZE = 512
LR = 1e-3
ITERS_PER_EVAL = 1000
ITERS_PER_LR_DECAY = 200
K = 100
LAMBDA = 1e-6

In [22]:
# setup
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
# device_name = 'cpu'
device = torch.device(device_name)
print(f"Using device {device}.")


model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

edge_index = uu_dataset.edge_index.to(device)

train_edge_index = train_edge_index.to(device)
train_sparse_edge_index = uu_dataset.train_sparse_edge_index.to(device)

val_edge_index = val_edge_index.to(device)
val_sparse_edge_index = uu_dataset.val_sparse_edge_index.to(device)

Using device cuda.


In [23]:
# training loop
train_losses = []
val_losses = []

for iter in tqdm(range(ITERATIONS)):

    # forward propagation
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(train_sparse_edge_index)

    # mini batching
    user_indices, pos_item_indices, neg_item_indices = uu_dataset.get_random_mini_batches(BATCH_SIZE)
    user_indices, pos_item_indices, neg_item_indices = user_indices.to(device), pos_item_indices.to(device), neg_item_indices.to(device)
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[neg_item_indices], items_emb_0[neg_item_indices]

    # loss computation
    train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                          pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if iter % ITERS_PER_EVAL == 0:
        model.eval()
        val_loss, recall, precision, ndcg = evaluation(
            model, val_edge_index, val_sparse_edge_index, [train_edge_index], K, LAMBDA)
        print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
        train_losses.append(train_loss.item())
        val_losses.append(val_loss)
        model.train()

    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
        scheduler.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

33703


  0%|          | 3/10000 [03:33<153:43:52, 55.36s/it] 

[Iteration 0/10000] train_loss: -0.69219, val_loss: -0.48347, val_recall@100: 0.01776, val_precision@100: 0.00091, val_ndcg@100: 0.00562


 10%|▉         | 999/10000 [04:41<10:01, 14.97it/s]  

...

 90%|████████▉ | 8999/10000 [40:56<01:08, 14.53it/s]   

33703


 90%|█████████ | 9003/10000 [44:17<5:51:19, 21.14s/it]

[Iteration 9000/10000] train_loss: -446.88718, val_loss: -43.8705, val_recall@100: 0.21698, val_precision@100: 0.01007, val_ndcg@100: 0.08166


100%|██████████| 10000/10000 [45:24<00:00,  3.67it/s] 
